In [1]:
import pandas as pd

df = pd.read_parquet("data")
print(df.head())

  carpark_id    carpark_name vendor  \
0      mitte  Parkhaus Mitte    MSR   
1      mitte  Parkhaus Mitte    MSR   
2      mitte  Parkhaus Mitte    MSR   
3      mitte  Parkhaus Mitte    MSR   
4      mitte  Parkhaus Mitte    MSR   

                                      current  
0  {'timestamp': 1728916440012, 'value': 218}  
1  {'timestamp': 1728916560015, 'value': 215}  
2  {'timestamp': 1728916680007, 'value': 209}  
3  {'timestamp': 1728916800007, 'value': 206}  
4  {'timestamp': 1728916920011, 'value': 200}  


In [2]:
current_df = pd.json_normalize(df["current"])
df = pd.concat([df.drop(columns=["current"]), current_df], axis=1)
df = df.rename(columns={"timestamp": "unix_time"})
df["ds"] = (
    pd.to_datetime(df["unix_time"], unit="ms", utc=True)
    .dt.tz_convert("Europe/Berlin")
)  
df = df.rename(columns={"value": "y"})
df["y"] = df["y"].astype(float)

df.head(10)

,carpark_id,carpark_name,vendor,unix_time,y,ds
0,mitte,Parkhaus Mitte,MSR,1728916440012,218.0,2024-10-14 16:34:00.012000+02:00
1,mitte,Parkhaus Mitte,MSR,1728916560015,215.0,2024-10-14 16:36:00.015000+02:00
2,mitte,Parkhaus Mitte,MSR,1728916680007,209.0,2024-10-14 16:38:00.007000+02:00
3,mitte,Parkhaus Mitte,MSR,1728916800007,206.0,2024-10-14 16:40:00.007000+02:00
4,mitte,Parkhaus Mitte,MSR,1728916920011,200.0,2024-10-14 16:42:00.011000+02:00
5,mitte,Parkhaus Mitte,MSR,1728917040004,201.0,2024-10-14 16:44:00.004000+02:00
6,mitte,Parkhaus Mitte,MSR,1728917160011,198.0,2024-10-14 16:46:00.011000+02:00
7,mitte,Parkhaus Mitte,MSR,1728917280004,196.0,2024-10-14 16:48:00.004000+02:00
8,mitte,Parkhaus Mitte,MSR,1728917760011,179.0,2024-10-14 16:56:00.011000+02:00
9,mitte,Parkhaus Mitte,MSR,1728917880007,177.0,2024-10-14 16:58:00.007000+02:00


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 655785 entries, 0 to 655784
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype                        
---  ------        --------------   -----                        
 0   carpark_id    655785 non-null  object                       
 1   carpark_name  655785 non-null  object                       
 2   vendor        655785 non-null  object                       
 3   unix_time     655785 non-null  int64                        
 4   y             655785 non-null  float64                      
 5   ds            655785 non-null  datetime64[ns, Europe/Berlin]
dtypes: datetime64[ns, Europe/Berlin](1), float64(1), int64(1), object(3)
memory usage: 30.0+ MB


In [4]:
import hopsworks
proj = hopsworks.login()
fs = proj.get_feature_store()

fg = fs.get_or_create_feature_group(name="parking", 
                                    version=1,
                                    primary_key=['carpark_id'], 
                                    event_time="ds",
                                    description="Parking occupancy data for Heilbronn"
                                   )
fg.insert(df)

2025-10-23 04:43:11,632 INFO: Python Engine initialized.

Logged in to project, explore it here https://hopsworks-stackit.hops.works/p/11384
Feature Group created successfully, explore it at 
https://hopsworks-stackit.hops.works/p/11384/fs/11332/fg/10259


Uploading Dataframe: 100.00% |██████████| Rows 655785/655785 | Elapsed Time: 00:12 | Remaining Time: 00:00


Launching job: parking_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://hopsworks-stackit.hops.works/p/11384/jobs/named/parking_1_offline_fg_materialization/executions


(Job('parking_1_offline_fg_materialization', 'SPARK'), None)